In [1]:
import urllib.request
from itertools import groupby

def Input(day):
    "Open input file for the day"
    file = '2017/input{0}.txt'.format(day)
    try:
        return open(file)
    except FileNotFoundError:
        return urllib.request.urlopen(
            'http:/adventofcode.com/2017/day/{0}/input'.format(day))
        

# Day 1: Inverse Captcha
The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

In [2]:
def parse(text):
    #append the first digit on the end, and group like digits
    return groupby(text + text[0])

def group(sequence):
    #return list of (digit, # of repetitions) tuples
    return [(int(digit),len(list(group))-1) for digit, group in parse(sequence)]

def inverse_captcha(groups):
    #sum all repeated digits
    return sum([digit*reps for digit,reps in groups if reps >= 1])

assert(inverse_captcha(group('1122')) == 3)
assert(inverse_captcha(group('1111')) == 4)
assert(inverse_captcha(group('1234')) == 0)
assert(inverse_captcha(group('91212129')) == 9)

inverse_captcha(group(Input(1).read().rstrip()))

1097

**Part 2**: Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

In [3]:
def parse(text):
    #pair digits and their shifted partners
    size = len(text)
    shift = get_shifter(size)
    return [(int(text[i]),int(text[shift(i)])) for i in range(size//2)]

def get_shifter(length):
    #given an index return the halfway shifted index
    return lambda x : (x + length // 2) % length

def halfway_captcha(values):
    return sum([i*2 for i,j in values if i == j])

assert(halfway_captcha(parse('1212')) == 6)
assert(halfway_captcha(parse('1221')) == 0)
assert(halfway_captcha(parse('123425')) == 4)
assert(halfway_captcha(parse('123123')) == 12)
assert(halfway_captcha(parse('12131415')) == 4)

halfway_captcha(parse(Input(1).read().rstrip()))


1188